In [14]:
import pandas as pd
import numpy as np
import json


json_file_1 = "./proj3_data1.json"
json_file_2 = "./proj3_data2.json"
json_file_3 = "./proj3_data3.json"

json_data_1 = pd.read_json(json_file_1)
json_data_2 = pd.read_json(json_file_2)
json_data_3 = pd.read_json(json_file_3)

df = pd.concat([json_data_1, json_data_2, json_data_3], ignore_index=True)
df.to_json('./proj3_ex01_all_data.json')

print(df)

          make     model  body_type  doors  top_speed  acceleration  \
0         Audi        A3  hatchback    5.0        220           7.2   
1          BMW  3 Series      sedan    4.0        240           6.2   
2   Volkswagen      Golf  hatchback    5.0        210           8.0   
3      Renault      Clio  hatchback    5.0        190          11.0   
4         Fiat       500  hatchback    3.0        160          12.9   
5      Peugeot       208  hatchback    NaN        190           9.9   
6        Volvo      XC60        SUV    5.0        220           8.0   
7         Seat      Leon  hatchback    5.0        240           8.0   
8         Opel     Corsa  hatchback    5.0        180           9.9   
9         Ford   Mustang      coupe    2.0        249           5.8   
10  Volkswagen    Passat      sedan    4.0        230           7.5   
11     Porsche       911      coupe    NaN        330           3.5   
12        Audi        Q5        SUV    5.0        237           6.3   
13    

In [24]:
df_copy = df.copy()

nans = df_copy.isna().sum()
nans_df = nans[nans > 0]

nans_df.to_csv('./proj3_ex02_no_nulls.csv')

In [16]:
with open('./proj3_params.json') as f:
    params = json.load(f)

print(params)

df_copy_second = df.copy()
print(df_copy_second)

df_copy_second['description'] = df_copy_second.apply(lambda row:  ' '.join(row[col] for col in params['concat_columns']), axis=1)

print(df_copy_second)
df_copy_second.to_json('./proj3_ex03_descriptions.json')

{'concat_columns': ['make', 'model', 'engine'], 'id_vars': ['make', 'model'], 'join_column': 'engine', 'int_columns': ['doors', 'displacement', 'horsepower', 'cylinders'], 'grouping_column': 'make', 'pivot_index': 'make', 'pivot_columns': 'fuel_type', 'pivot_values': 'fuel_consumption', 'aggregations': [['displacement', 'min'], ['displacement', 'max'], ['fuel_consumption', 'mean']]}
          make     model  body_type  doors  top_speed  acceleration  \
0         Audi        A3  hatchback    5.0        220           7.2   
1          BMW  3 Series      sedan    4.0        240           6.2   
2   Volkswagen      Golf  hatchback    5.0        210           8.0   
3      Renault      Clio  hatchback    5.0        190          11.0   
4         Fiat       500  hatchback    3.0        160          12.9   
5      Peugeot       208  hatchback    NaN        190           9.9   
6        Volvo      XC60        SUV    5.0        220           8.0   
7         Seat      Leon  hatchback    5.0    

In [17]:
df_more_data = pd.read_json('./proj3_more_data.json')
print(df_more_data)

df_from_3rd = df_copy_second.copy()
# df_from_3rd.merge(df_more_data[params['join_column']], how='left')
data_joined = df_from_3rd.merge(df_more_data, on=params["join_column"], how="left")
data_joined.to_json("proj3_ex04_joined.json")

                    engine  displacement  horsepower fuel_type  cylinders  \
0                 1.5 TFSI          1498         150  gasoline          4   
1     2.0L TwinPower Turbo          1998         184  gasoline          4   
2                  1.5 TSI          1498         150  gasoline          4   
3                  1.5 dCi          1461         115    diesel          4   
4              0.9 TwinAir           875          85  gasoline          2   
5             1.2 PureTech          1199         130  gasoline          3   
6                1.2 Turbo          1197         110  gasoline          4   
7            2.3L EcoBoost          2261         350  gasoline          4   
8                  2.0 TDI          1968         150    diesel          4   
9       3.0L Twin Turbo V6          3000         450  gasoline          6   
10         45 TFSI quattro          1984         245  gasoline          4   
11  T8 Polestar Engineered          1969         415    hybrid          4   

In [18]:
import json
df_from_4th = df_from_3rd.copy()

cars = df_from_4th.to_dict(orient='records')
for car in cars:
    file_name = car['description']
    file_name = file_name.replace(" ", "_").lower()
    del car['description']
    json_string = json.dumps(car)
    with open('./proj3_ex05_' + file_name + '.json', 'w') as f:
        f.write(json_string)
    
df_from_4th.where(pd.notnull(df_from_4th), None)
for column in params['int_columns']:
    if column in df_from_4th.columns:
        df_from_4th[column].astype(float).astype(pd.Int64Dtype())

cars_int = df_from_4th.to_dict(orient='records')
for car in cars_int:
    file_name = car['description']
    file_name = file_name.replace(" ", "_").lower()
    del car['description']
    json_string = json.dumps(car)
    with open('./proj3_ex05_int_' + file_name + '.json', 'w') as f:
        f.write(json_string)

In [19]:
df_ex6={}
for t in params['aggregations']:
    col = t[0]
    ff = t[1]
    df_ex6[f'{ff}_{col}'] = data_joined.agg({col : ff})[col]
with open('proj3_ex06_aggregations.json', 'w') as file:
    json.dump(df_ex6, file)

In [20]:
group_col = params['grouping_column']
group = data_joined.groupby(group_col).filter(lambda x:len(x)>1)
group = group.groupby(group_col).mean(numeric_only=True)
group.to_csv('proj3_ex07_groups.csv', header=True, index=True)

In [21]:
import pickle
topkl = data_joined.pivot_table(index=params['pivot_index'], columns=params['pivot_columns'], values=params['pivot_values'], aggfunc='max')
with open('proj3_ex08_pivot.pkl', 'wb') as file:
    pickle.dump(topkl, file)



In [22]:
result = data_joined.melt(id_vars=params['id_vars'])
result.to_csv('proj3_ex08_melt.csv', header=True, index=False)
df_ex8=pd.read_csv('proj3_statistics.csv')
pref = set(df_ex8.columns[1:].str.split('_').str[0].unique().to_list())
data_result = pd.wide_to_long(df_ex8, pref, i='Country',j='Year', sep='_')
data_result.index = data_result.index.to_series()

with open('proj3_ex08_stats.pkl', 'wb') as file:
    pickle.dump(data_result, file)